<a href="https://colab.research.google.com/github/Gilf641/EVA4/blob/master/S5_AssignmentSolution3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the required libraries

from __future__ import print_function
import torch # PyTorch Main Library
import torch.nn as nn # Neural Net Lib
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms 
import matplotlib.pyplot as plt


In [2]:
# Data Loading and Pre-Processing


  
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")




torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),                          
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


test_loader = torch.utils.data.DataLoader(

    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)





In [3]:
# Model 6 - Complete different architecture bcoz I'm stuck between 99-99.2%. Maybe my net's not that deep, I'm trying something new in this architecture
# set the dropout value

class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()


     # Conv Block1
      self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(8)) # output_size = 26
            

    # Conv Block2 
      self.convblock2 = nn.Sequential(
        nn.Conv2d(in_channels = 8, out_channels = 12, kernel_size = (3,3), padding=0, bias=False),
        nn.ReLU(),
        nn.BatchNorm2d(12)) # output_size = 24  



    # Conv Block3
      self.convblock3 = nn.Sequential(
          nn.Conv2d(in_channels = 12, out_channels = 12, kernel_size=(3,3), padding=0, bias=False),
          nn.ReLU(),
          nn.BatchNorm2d(12)) # output_size = 22


    # TRANSITION BLOCK 1
      self.pool1 = nn.Sequential(nn.MaxPool2d(2, 2))# output_size = 11


    # Conv Block4
      self.convblock4 = nn.Sequential(
        nn.Conv2d(in_channels = 12, out_channels = 16, kernel_size = (1,1), padding=0, bias=False),
        nn.ReLU(),
        nn.BatchNorm2d(16)) # output_size = 11


    # Conv Block5
      self.convblock5 = nn.Sequential(
        nn.Conv2d(in_channels = 16, out_channels = 16, kernel_size = (3,3), padding=0, bias=False),
        nn.ReLU(),
        nn.BatchNorm2d(16)) #output_size = 9


    # Conv Block6
      self.convblock6 = nn.Sequential(
          nn.Conv2d(in_channels = 16, out_channels = 16, kernel_size=(3,3), padding = 0, bias=False),
          nn.ReLU(),
          nn.BatchNorm2d(16)) # output_size = 7

 
    # Conv Block7
      self.convblock7 = nn.Sequential(
          nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3,3), padding = 0, bias = False),
          nn.ReLU(),
          nn.BatchNorm2d(16)) # output_size = 5


    # Conv Block8
      self.convblock8 = nn.Sequential(
        nn.Conv2d(in_channels = 16, out_channels=10, kernel_size =(1,1), padding =0, bias=False)) # output_size = 1
      

    # Output Block
      self.gap = nn.Sequential(nn.AvgPool2d(5)) # Global Average Pooling


    # DropOut Layer
      self.drop1 = nn.Sequential(nn.Dropout(0.05))


    def forward(self, x):
      x = self.convblock1(x)
      x = self.convblock2(x)
      x = self.convblock3(x)
      x = self.pool1(x)
      x = self.convblock4(x)
      x = self.convblock5(x)
      x = self.convblock6(x)
      x = self.drop1(x)
      x = self.convblock7(x)
      x = self.gap(x)
      x = self.convblock8(x)

    
      
      x = x.view(-1, 10)
      return F.log_softmax(x, dim=-1)


model = Net().to(device)
summary(model, input_size=(1, 28, 28))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4           [-1, 12, 24, 24]             864
              ReLU-5           [-1, 12, 24, 24]               0
       BatchNorm2d-6           [-1, 12, 24, 24]              24
            Conv2d-7           [-1, 12, 22, 22]           1,296
              ReLU-8           [-1, 12, 22, 22]               0
       BatchNorm2d-9           [-1, 12, 22, 22]              24
        MaxPool2d-10           [-1, 12, 11, 11]               0
           Conv2d-11           [-1, 16, 11, 11]             192
             ReLU-12           [-1, 16, 11, 11]               0
      BatchNorm2d-13           [-1, 16, 11, 11]              32
           Conv2d-14             [-1, 1

In [0]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nEPOCH:{} \nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [5]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 16
for epoch in range(1, 16):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Loss=0.165153369307518 Batch_id=468 Accuracy=90.87: 100%|██████████| 469/469 [01:12<00:00,  6.49it/s]
Loss=0.07207398861646652 Batch_id=0 Accuracy=97.66:   0%|          | 1/469 [00:00<01:21,  5.72it/s]


EPOCH:1 
Test set: Average loss: 0.1201, Accuracy: 9654/10000 (96.54%)



Loss=0.042833179235458374 Batch_id=468 Accuracy=98.04: 100%|██████████| 469/469 [01:13<00:00,  6.38it/s]
Loss=0.06611917912960052 Batch_id=0 Accuracy=99.22:   0%|          | 1/469 [00:00<01:14,  6.31it/s]


EPOCH:2 
Test set: Average loss: 0.0435, Accuracy: 9875/10000 (98.75%)



Loss=0.08959776908159256 Batch_id=468 Accuracy=98.45: 100%|██████████| 469/469 [01:13<00:00,  6.38it/s]
Loss=0.03741735592484474 Batch_id=0 Accuracy=99.22:   0%|          | 1/469 [00:00<01:15,  6.22it/s]

...


EPOCH:13 
Test set: Average loss: 0.0214, Accuracy: 9936/10000 (99.36%)



Loss=0.08255953341722488 Batch_id=468 Accuracy=99.25: 100%|██████████| 469/469 [01:12<00:00,  6.49it/s]
Loss=0.04812057316303253 Batch_id=0 Accuracy=98.44:   0%|          | 1/469 [00:00<01:09,  6.71it/s]


EPOCH:14 
Test set: Average loss: 0.0195, Accuracy: 9937/10000 (99.37%)



Loss=0.03281335160136223 Batch_id=468 Accuracy=99.31: 100%|██████████| 469/469 [01:11<00:00,  6.53it/s]



EPOCH:15 
Test set: Average loss: 0.0197, Accuracy: 9940/10000 (99.40%)

